# Table 1
This file produces the results from "Table 1. Poverty in Fruitvale". 

In [4]:
#Import packages
import pandas as pd
import matplotlib as mpl
import numpy as np
import math
import os
pd.options.mode.chained_assignment = None #gets rid of warning for chained alterations

In [5]:
#set directories
os.chdir("/Users/briangoggin/Dropbox/CP 201A/Fruitvale")
cwd = os.getcwd()
root = cwd #root is directory to raw files

In [6]:
#set input data source
poverty = root+"/Raw Data/ACS_14_5YR_B17001_with_ann.csv" 


In [7]:
#identify Fruitvale Census Tracts
Fruitvale = [6001406100, 6001406201, 6001406202, 6001406300, 6001406500, 6001407101, 6001407102, 6001407200, 6001407300, 6001407400, 6001407500, 6001407600]


In [8]:
#Define function for import and standard ACS cleaning operations
def import_census(file):
    df = pd.read_csv(file, header = 1)
    df['Fruitvale'] = df['Id2'].isin(Fruitvale)
    #the following 5 lines create duplicate observations of Fruitvale to include them in county averages
    df2 = df[df['Fruitvale'] == True]
    df2.drop('Fruitvale', axis = 1)
    df2.is_copy = False
    df2['Fruitvale'] = False
    df = df.append(df2)
    return df

#Define function for proportions MOE
def prop_MOE(numerator, denominator, num_moe, den_moe):
    return ((num_moe**2 - (((numerator/denominator)**2)*(den_moe**2))))**(1/2)/denominator

In [9]:
#import data
pov = import_census(poverty)

In [10]:
#rename variables
pov['total'] = pov["Estimate; Total:"].astype(int)
pov['total_moe'] = pov["Margin of Error; Total:"].astype(int)
pov['poor'] = pov["Estimate; Income in the past 12 months below poverty level:"].astype(int)
pov['poor_moe'] = pov["Margin of Error; Income in the past 12 months below poverty level:"].astype(int)

#convert MOEs to 95% confidence interval
pov['total_moe'] = pov['total_moe']*(1.96/1.645)
pov['poor_moe'] = pov['poor_moe']*(1.96/1.645)

#prepare variables for collapse
pov['total_moe2'] = pov['total_moe']**2
pov['poor_moe2'] = pov['poor_moe']**2

In [11]:
#sum data
sumvars = ['total', 'total_moe2', 'poor', 'poor_moe2']
pov_sum = pov.groupby('Fruitvale')[sumvars].sum()
pov_sum['total_moe'] = pov_sum['total_moe2']**(1/2)
pov_sum['poor_moe'] = pov_sum['poor_moe2']**(1/2)
pov_sum['pct_poor'] = (pov_sum['poor']/pov_sum['total'])
pov_sum['pct_poor_moe'] = prop_MOE(pov_sum['poor'], pov_sum['total'],  pov_sum['poor_moe'], pov_sum['total_moe'])

In [12]:
#final cleaning
pov_sum = pov_sum.round(4)
pov_sum['pct_poor'] = 100*pov_sum['pct_poor']
pov_sum['pct_poor_moe'] = 100*pov_sum['pct_poor_moe']


In [13]:
# Export file for table creation 
export_path = cwd+'/Output/python_output/pov_finalproject.csv'
pov_sum.to_csv(export_path)